# SRTM data from Rasdaman

## Important note
The diagnosed bug in the data is **not** a bug. It is rather a product of a bug in the plotting routine (`create_mapplot`) which seems to be unsuitable for the coordinate ordering of the data at hand. This was noticed when the 'corrected' data was processed with CDO after which the data proved to be distorted. <br>
The only thing to accomplish with the netCDF-file from Rasdaman is to attribute some attributes for the underlying coordinates. Specifically, the follwoing was done to get the desired file (`/p/project/deepacf/maelstrom/data/ap5/downscaling_ifs2radklim/srtm_data/topography_srtm_ifs2radklim.nc`). 
* cp topography_srtm.nc topography_srtm_cf.nc
* ncatted -a axis,Lat,a,c,Y topography_srtm_cf.nc
* ncatted -a standard_name,Lat,a,c,latitude topography_srtm_cf.nc
* ncatted -a units,Lat,a,c,degrees_north topography_srtm_cf.nc
* ncatted -a units,Lon,a,c,degrees_east topography_srtm_cf.nc
* ncatted -a standard_name,Lon,a,c,longitude topography_srtm_cf.nc
* ncatted -a axis,Lon,a,c,X topography_srtm_cf.nc
* cdo -L setmisstoc,-0.5 -chname,Gray,surface_elevation topography_srtm_cf.nc topography_srtm_cf_oceanic.nc
* cdo -L remapcon,ifs2radklim_grid topography_srtm_cf_oceanic.nc test2.nc
* cdo -f nc2 copy topography_srtm_ifs2radklim.nc topography_srtm_ifs2radklim_nc2.nc
* ncrename -d lon,lon_tar -v lon,lon_tar -d lat,lat_tar -v lat,lat_tar topography_srtm_ifs2radklim_nc2.nc
* ncatted -a units,surface_elevation,o,c,m topography_srtm_ifs2radklim_nc2.nc
* cdo -O -f nc4 copy test.nc topography_srtm_ifs2radklim.nc

Note that the conversion to netCDF2 is required to avoid broken coordinate values after renaming the dimensions (see [here](https://sourceforge.net/p/nco/discussion/9830/thread/44038f3bda/)).

## Notebook start (bug!)
We start by loading some modules...

In [ ]:
%matplotlib inline
    
# import modules
import os, sys
import datetime as dt
import numpy as np
import xarray as xr
import pandas as pd
# for plotting
import matplotlib as mpl
import matplotlib.cm                # needed on some machines... why ever?!
import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs

... and creating some auxiliary functions to plot the data.

In [ ]:
def get_cmap_norm(col_obj, levels):
    """
    Obtain norm from colormap with desired levels
    :param levels: level boundaries
    :return cmap: colormap-object
    :return norm: normalization object corresponding to colormap and levels
    """
    bounds = np.asarray(levels)

    nbounds = len(bounds)

    # create colormap and corresponding norm
    cmap = mpl.colors.ListedColormap(col_obj, name="temp" + "_map")
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    return cmap, norm


def decorate_mapplot(ax_plot, plot_xlabel=True, plot_ylabel=True, **kwargs):
    """
    Make a map-plot nicely looking by adding coastlines, country borders and also setting up the plot axes.
    """
    fs = kwargs.get("fs", 16) 
    coast_lw = kwargs.get("coast_lw", 0.75)
    country_lw = kwargs.get("country_lw", 0.75)
    domain_extent = kwargs.get("dom_extent", [3.5, 16.5, 44.5, 54.]) 
    tick_spacing = kwargs.get("tick_spacing", 5.)
    

    ax_plot.coastlines(linewidth=coast_lw)
    ax_plot.coastlines(linewidth=coast_lw)
    ax_plot.add_feature(cartopy.feature.BORDERS, linewidth=country_lw)

    # adjust extent and ticks as well as axis-label
    ax_plot.set_xticks(np.arange(0., 360. + 0.1, tick_spacing))  # ,crs=projection_crs)
    ax_plot.set_yticks(np.arange(-90., 90. + 0.1, tick_spacing))  # ,crs=projection_crs)

    ax_plot.set_extent(domain_extent)#, crs=prj_crs)
    ax_plot.minorticks_on()
    ax_plot.tick_params(axis="both", which="both", direction="out", labelsize=fs-2)

    # some labels
    if plot_xlabel:
        ax_plot.set_xlabel("Longitude [°E]", fontsize=fs)
    if plot_ylabel:
        ax_plot.set_ylabel("Latitude [°N]", fontsize=fs)

    return ax_plot


def create_mapplot(data: xr.DataArray, plt_fname: str, **kwargs):
    """
    Plot geo-referenced data onto a map.
    :param data: the geo-referenced data-array (xr.DataArray) to be plotted
    :param plt_fname: name of plot-file to be created 
    :param kwargs: optional keyword arguments. Known keyword arguments are:
                   spatial_dims: list of dimension names for spatial dimensions (default: ["lat", "lon"])
                   dom_extent: domain extent where the four elements represent the domain corner in
                               [west, east, south, north] direction (default: [3.5, 17.5, 44.5, 55.]) 
                   levels: levels-array for colour encoding the data (default: np.arange(-5., 25.1, 1.))
                   title: Title of the plot which is extended by a time stamp (default: "")
                   coast_lw: line width of coast lines on the map (default: 0.75)
                   tick_spacing: spacing of x-/y-axis ticks in degree (default: 2.)
                   projection_map: projection of the resulting map (default: ccrs.PlateCarree())
                   fs: font size of title, axis-labels etc., fs of legend and tick will be smaller by 2 (default: 16)
                   cmap: colormap-object obtained with mpl.cm.get_cmap (default: based on colormap "PuOr_r")
                   
    """
    # get optional keyword arguments or set default
    spatial_dims = kwargs.get("spatial_dims", ["lat", "lon"])
    domain_extent = kwargs.get("dom_extent", [3.5, 17.5, 44.5, 55.]) 
    levels = kwargs.get("levels", np.arange(-5., 25.1, 1.))
    title = kwargs.get("title", "")
    coast_lw = kwargs.get("coast_lw", 0.75)
    tick_spacing = kwargs.get("tick_spacing", 2.)
    data_unit = kwargs.get("unit", "")
    projection_map = kwargs.get("projection_map", ccrs.PlateCarree())
    projection_data = kwargs.get("projection_data", ccrs.PlateCarree())
    fs = kwargs.get("fs", 16)
    # finally obtain colormap-object which requires levels-information
    lvl = np.asarray(levels)
    nbounds = len(lvl)
    cmap = kwargs.get("cmap", mpl.cm.get_cmap("PuOr_r")(np.linspace(0., 1., nbounds))) 

    # get coordinates and time stamp from data
    try:
        lon, lat = data[spatial_dims[0]].values, data[spatial_dims[1]].values
    except Exception as err:
        print("Failed to retrieve coordinates from data1")
        raise err
    # construct array for edges of grid points
    dy, dx = np.round((lat[1] - lat[0]), 7), np.round((lon[1] - lon[0]), 7)
    lat_e, lon_e = np.arange(lat[0]-dy/2, lat[-1]+dy, dy), np.arange(lon[0]-dx/2, lon[-1]+dx, dx)

    title = "{0}".format(title)
    
    print("To verify retrieved coordinates:")
    print(f"Latitude of grid edges: {lat_e}")
    print(f"Longitude of grid edges: {lon_e}")
    print(data)

    # get colormap and corresponding norm
    cmap, norm = get_cmap_norm(cmap, levels)
    # create plot objects
    fig, ax = plt.subplots(1, 1, figsize=(9, 6), subplot_kw={"projection": projection_map})

    # perform plotting
    map_plot = ax.pcolormesh(lon_e, lat_e, np.squeeze(data.values), cmap=cmap, norm=norm,
                             transform=projection_data)

    # make plot nice
    ax = decorate_mapplot(ax, fs=fs, coast_lw=coast_lw, dom_extent=domain_extent, tick_spacing=tick_spacing)

    ax.set_title(title, size=fs) 

    # add colorbar
    cax = fig.add_axes([0.9, 0.2, 0.02, 0.6])
    cbar = fig.colorbar(map_plot, cax=cax, orientation="vertical", ticks=lvl[1::2])
    cbar.ax.tick_params(labelsize=fs-2)
    
    # add unit-string
    fig.text(0.92, 0.82, data_unit, fontsize=fs, horizontalalignment='center')
    
    # show and save plot to file
    print(f"Plot data to file '{plt_fname}.png'")
    
    plt.show()
    fig.savefig(plt_fname+".png", bbox_inches="tight")
    plt.close(fig)

Next, we read the retrieved SRTM datafile from disk:

In [ ]:
data_dir = "/p/project/deepacf/maelstrom/data/"
fname_in = "topography_srtm.nc"

# construct filenames
fname_in = os.path.join(data_dir, fname_in)
fname_out = fname_in.replace(".nc", "_corr.nc")

# open datafile
topo_ds = xr.open_dataset(fname_in)

Lets's have a brief look on the dataset

In [ ]:
topo_ds["Gray"]

As the dataset is huge, we slice it to a smaller region for visulaization purposes:

In [ ]:
ncrop = 3000    # number of data points in lon- and lat-direction 
ist = 7000      # start index for slicing
topo_ds_crop = topo_ds.isel({"Lon": slice(ist, ist+ncrop), "Lat": slice(ist, ist+ncrop)})

In [ ]:
print(topo_ds_crop)

Then, we plot the data as it is after customizing the plot apparance.

In [ ]:
plt_name = fname_in.replace(".nc", "_orig")

levels = [-100., 2., 10, 20, 30., 50., 70., 100., 150., 200., 250., 300., 400., 500., 1000.]
cmap_obj = mpl.cm.get_cmap("terrain")(np.linspace(0.2, 1., len(levels)))
dom_extent = [5.5, 8.5, 51.5, 54.5]

create_mapplot(topo_ds_crop["Gray"], plt_name, levels=levels, cmap=cmap_obj, title="surface elevation",
               unit="m", spatial_dims=["Lon", "Lat"] ,dom_extent=dom_extent)

It's easily seen that the topograohy data looks weird (e.g. surface elevation of more than 50 m over the North Sea). However, it looks like that one just needs to rotate the contour plot by 90° in clockwise direction. <br>
In the following, we will verify this:

In [ ]:
spatial_dims = ["lon", "lat"]
# transpose the data while keeping the coordinate data as it is (apart from slightly renaming them)
topo_da_new = xr.DataArray(topo_ds_crop["Gray"].values.transpose(), dims=spatial_dims,
                           coords={"lon": ("lon", topo_ds_crop["Lon"].values), 
                                   "lat": ("lat", topo_ds_crop["Lat"].values)}, 
                           attrs={"units": "m"}, name="surface_elevation")


Let's plot the transposed data array:

In [ ]:
create_mapplot(topo_da_new, plt_name.replace("_orig", "_transpose"), levels=levels, cmap=cmap_obj, title="surface elevation",
               unit=topo_da_new.attrs["units"], spatial_dims=spatial_dims, dom_extent=dom_extent)

Et voila - That's looks much better!

Finally, let's correct the data for further processing:

In [ ]:
a= topo_ds["Gray"]
print(a.values)

To be able to transpose the data, we need to slice quadratic fields which must at the same time cover the target region. 
As can be deduced from the Figure below, the data must at least cover lon=[4..16.5] and lat=[46.5, 55.]. 
This is ensured with the following slicing on the data.

![RADKLIM region](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTYuF895rjXlWKVVekTjQ_6gz7fQNRLfDTkdQ&usqp=CAU)

In [ ]:
ncrop = 24000
nstart = 4800

topo_data = topo_ds["Gray"].isel({"Lon": slice(nstart, nstart+ncrop)}).copy()

topo_data = topo_data.fillna(-9999.)
land_mask = xr.where(topo_data == -9999., 0, 1)
topo_data = topo_data.where(topo_data > -9998., -0.5)

Next, we transpose the array:

In [ ]:
topo_data_new = xr.DataArray(topo_data.values.transpose(), dims=["lon", "lat"],
                             coords={"lon": ("lon", topo_data["Lon"].values), 
                                     "lat": ("lat", topo_data["Lat"].values)}, 
                             attrs={"units": "m"}, name="surface_elevation")
land_mask_new = xr.DataArray(land_mask.values.transpose(), dims=["lon", "lat"],
                             coords={"lon": ("lon", topo_data["Lon"].values), 
                                     "lat": ("lat", topo_data["Lat"].values)}, 
                             attrs={"units": "m"}, name="surface_elevation")

topo_data_new, land_mask_new = topo_data_new.transpose("lat", "lon"), land_mask_new.transpose("lat", "lon")

ds_new = xr.Dataset({"surface_elevation": topo_data_new, "land_sea_mask": land_mask_new})

In [ ]:
print(ds_new)

We also add some coordinate attributes to fulfill CF conventions (needed to process the data with CDO!).

In [ ]:
ds_new["lon"].attrs["standard_name"] = "longitude"
ds_new["lon"].attrs["units"] = "degrees_east"

ds_new["lat"].attrs["standard_name"] = "latitude"
ds_new["lat"].attrs["units"] = "degrees_north"

Finally, we write the dataset into a netCDF-file.

In [ ]:
if not os.path.isfile(fname_out):
    print(f"Write dataset (back) to netCDF-file '{fname_out}'")
    ds_new.to_netcdf(fname_out)
else:
    print(f"File '{fname_out}' already exists. Remove it in case you want to obtain a new one.")

In [ ]:
nstart = 5500
ncrop = 5000

levels = [-100., 0., 10., 30, 50., 100., 200., 300., 500., 750., 1000., 1250., 1500., 2000., 2500.]

create_mapplot(topo_data_new.isel({"lon": slice(nstart, nstart+ncrop), "lat": slice(nstart, nstart+ncrop)}), plt_name.replace("_orig", "_check"), levels=levels, cmap=cmap_obj, title="surface elevation",
               unit="m", spatial_dims=["lon", "lat"] ,dom_extent=[8., 13., 55.5, 51.])

In [ ]:
remapped_file = os.path.join(data_dir, "topography_srtm_ifs2radklim.nc")

data_rem = xr.open_dataset(remapped_file)

In [ ]:
print(data_rem)

In [ ]:
plt_name = os.path.join(data_dir, "remapped_test")

create_mapplot(data_rem["surface_elevation"], plt_name, levels=levels, cmap=cmap_obj, title="surface elevation",
               unit="m", spatial_dims=["lon", "lat"] ,dom_extent=[8., 13., 55.5, 51.])